In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import tensorflow as tf
import cv2
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
from skimage.filters.rank import equalize
from skimage.morphology import disk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import glob, os
# os.chdir("/content/drive/MyDrive/EECS-6893-Project/CASIA-test")
# count = 0 
# for filename in glob.glob("*.jpg"):
#     person_name = "/content/drive/MyDrive/EECS-6893-Project/CASIA-test" + "/" + filename
#     image = cv2.imread(person_name)
#     cv2.imwrite("/content/drive/MyDrive/EECS-6893-Project/CASIA-clean/" + filename, image)
#     image_path = "/content/drive/MyDrive/EECS-6893-Project/CASIA-clean" + "/" + filename
    image = Image.open(image_path)
    # print("Image size: ", np.asarray(image).shape)
    # print(count)
    # count = count + 1

In [ ]:
directory_name = "/content/drive/MyDrive/EECS-6893-Project/CASIA-clean"
x, y = np.zeros([2838, 224, 224, 3]), np.zeros([2838])


from collections import defaultdict
index = 0
n = 0
record = set()
dic = defaultdict(int)
for filename in os.listdir(directory_name):
  image = Image.open(directory_name + '/' + filename) #open image

  checker = filename[:5]

  if checker not in record:
    record.add(checker)
    n += 1

  dic[str(checker)] += 1

  image = image.resize((224,224)) # resize 
  image = np.asarray(image) # convert to array

  x[index,:,:,:] = image
  y[index] = n

  flip = np.flip(x[index],1)
  flip = np.resize(flip, (1,224,224,3))
  x[index+1419,:,:,:] = flip
  y[index+1419] = n
  index += 1



In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(len(x_train), len(x_test))

2270 568


In [ ]:
len(record)

218

In [ ]:
index_train = np.random.permutation(len(x_train))
index_test = np.random.permutation(len(x_test))
x_train, y_train = x_train[index_train], y_train[index_train]
x_test, y_test = x_test[index_test], y_test[index_test]

In [ ]:
x_train, x_test = tf.cast(x_train, tf.float32)/255.0, tf.cast(x_test, tf.float32)/255.0
y_train, y_test = tf.one_hot(y_train, depth=218), tf.one_hot(y_test, depth=218)

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import Model

#from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten

In [ ]:
from tensorflow.keras import Model
pre_trained_model = ResNet50(input_shape=(224,224,3), weights='imagenet')
ResNet = Model(inputs=pre_trained_model.input,outputs=pre_trained_model.layers[-2].output)

# First time, train without adjusting the ResNet weights to train the final dense layer first
ResNet.trainable =  True
model = tf.keras.Sequential([
                             ResNet,
                             tf.keras.layers.Dense(218, activation='softmax')
                                      ])
model.summary()

# from tensorflow.keras import Model
# pre_trained_model = VGG16(input_shape=(224,224,3), weights='imagenet')
# vgg_model = Model(inputs=pre_trained_model.input,outputs=pre_trained_model.layers[-2].output)


# # First time, train without adjusting the ResNet weights to train the final dense layer first
# vgg_model.trainable =  False
# model = tf.keras.Sequential([
#                              vgg_model,
#                              Flatten(),
#                              tf.keras.layers.Dense(512, activation='relu'),
#                              tf.keras.layers.Dense(190, activation='softmax')
#                                       ])
# model.summary()

model.add_loss(lambda: 0.01*tf.reduce_sum(tf.square(model.layers[-1].kernel)))

model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=tf.keras.metrics.CategoricalAccuracy())

model.fit(x_train, y_train, batch_size=24, epochs=1, validation_split=0.2)

ResNet.trainable =  True
model.add_loss(lambda: 0.01*tf.reduce_sum(tf.square(model.layers[-1].kernel)))

model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
              metrics=tf.keras.metrics.CategoricalAccuracy())

model.fit(x_train, y_train, batch_size=24, epochs=40, validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_3 (Functional)        (None, 2048)              23587712  
                                                                 
 dense_3 (Dense)             (None, 218)               446682    
                                                                 
Total params: 24,034,394
Trainable params: 23,981,274
Non-trainable params: 53,120
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


76/76 [==============================] - 20s 168ms/step - loss: 7.4749 - categorical_accuracy: 0.2786 - val_loss: 8.8754 - val_categorical_accuracy: 0.0088
Epoch 1/40
76/76 [==============================] - 19s 167ms/step - loss: 6.8920 - categorical_accuracy: 0.7004 - val_loss: 9.9414 - val_categorical_accuracy: 0.0022
Epoch 2/40
76/76 [==============================] - 12s 152ms/step - loss: 4.4525 - categorical_accuracy: 0.8673 - val_loss: 8.8510 - val_categorical_accuracy: 0.0022
Epoch 3/40
76/76 [==============================] - 12s 152ms/step - loss: 3.1978 - categorical_accuracy: 0.9334 - val_loss: 7.9360 - val_categorical_accuracy: 0.0022
Epoch 4/40
76/76 [==============================] - 12s 152ms/step - loss: 2.3749 - categorical_accuracy: 0.9559 - val_loss: 7.3188 - val_categorical_accuracy: 0.0022
Epoch 5/40
76/76 [==============================] - 12s 152ms/step - loss: 1.8225 - categorical_accuracy: 0.9752 - val_loss: 6.8813 - val_categorical_accuracy: 0.0022
Epoch 6/4

In [ ]:
model.evaluate(x_test, y_test)

18/18 [==============================] - 1s 58ms/step - loss: 9.0651 - categorical_accuracy: 0.7025


[9.06507682800293, 0.702464759349823]

In [ ]:
pred = model.predict(x_test)
np.mean(np.argmax(y_test, axis=1)==np.argmax(pred, axis=1))

0.7024647887323944